In [4]:
#1
import pandas as pd
import matplotlib.pyplot as plt
#import torch
import json

datetime_data = "1009Data"
data_path = f"{datetime_data}/ap2/"
label_dic = {"1p":1,"2p":2,"3p":3}

data_label = ["time", "type", "role", "mac", "rssi", "rate", "sig_mode", "mcs", "bandwidth", "smoothing",
            "not_sounding", "aggregation", "stbc", "fec_coding", "sgi", "noise_floor", "ampdu_cnt", 
            "channel", "secondary_channel", "local_timestamp", "ant", "sig_len", "rx_state", "real_time_set",
            "real_timestamp", "len", "CSI_DATA"]

def CSI_data_parser(csi_data):
    num = 0 
    output_lst = [] 
    for idx in range(len(csi_data.index)):
        CSI_output = csi_data.loc[idx]
        CSI_output_lst = [int(i) for i in CSI_output[2:-4].split(" ") if i.lstrip('-').isdigit()]

        #print(" ".join(map(str, CSI_output_lst))) #除錯
        output_lst.append(CSI_output_lst)
        try:
            assert len(CSI_output_lst) == 127
        except:
            print(len(CSI_output_lst))
    return output_lst


# f"{datetime_data}/20240503label.json"
def get_label(path):
    with open(path, "r") as f:
        label_data = json.load(f)
    return label_data

def label_parser(label_dic, data_num):
    for k in label_dic.keys():
        data = label_dic[k]

        s = pd.to_datetime("10/09 " + data[0], format='%m/%d %H:%M:%S.%f')
        e = pd.to_datetime("10/09 " + data[1], format='%m/%d %H:%M:%S.%f')
        
        label_lst.append([s, e])
    return label_lst

### get datalabel
---

In [5]:
#2
#把資料中不要的欄位去掉並標記時間

data_num = range(1, 11)  # data_num +1

output_num = 1
#標記資料
label_json = get_label(f"1009Data/1009_complete_label_2p.json")

for data_id in data_num:
    label_lst = []
    print("dataid=>", data_id)
    #CSI資料
    data = pd.read_csv(f"1009Data/ap2/filtered_20241009AP2WALK_2P.csv")

    # 初始化標籤欄位
    data["label"] = 0

    label_lst = label_parser(label_json["data1"]["2p"], data_id)
    print("label_lst", label_lst)
    
    def trns(x):
        for idx, lst in enumerate(label_lst):
            if x > lst[0] and x < lst[1]:
                return 3
        return 0  # 其他標籤
    
    data["time"] = pd.to_datetime(data["time"], format='%m/%d %H:%M:%S.%f')
    
    # 應用標籤範圍
    data["label"] = data.apply(lambda row: trns(row["time"]) if row["label"] == 0 else row["label"], axis=1)
    print(data.columns)

    # 解析 CSI_DATA 欄位
    csi_data = CSI_data_parser(data["CSI_DATA"])
    out_csi = pd.DataFrame(csi_data, columns=[f"c{i}" for i in range(127)])
    
    # 複製時間和 MAC 地址欄位
    out_csi["time"] = data["time"].dt.strftime('%m/%d %H:%M:%S.%f')
    out_csi["mac"] = data["mac"].copy()
    out_csi["label"] = data["label"].copy()
    
    # 將結果輸出到新的 CSV 文件中
out_csi.to_csv(f"{datetime_data}/Clean_data_MAC/test2.csv", index=False)

dataid=> 1
label_lst [[Timestamp('1900-10-09 22:49:37.516000'), Timestamp('1900-10-09 22:49:39.034000')], [Timestamp('1900-10-09 22:49:54.874000'), Timestamp('1900-10-09 22:49:56.623000')], [Timestamp('1900-10-09 22:50:11.274000'), Timestamp('1900-10-09 22:50:12.827000')], [Timestamp('1900-10-09 22:50:26.982000'), Timestamp('1900-10-09 22:50:28.831000')], [Timestamp('1900-10-09 22:50:38.862000'), Timestamp('1900-10-09 22:50:40.777000')], [Timestamp('1900-10-09 22:50:56.913000'), Timestamp('1900-10-09 22:50:58.664000')], [Timestamp('1900-10-09 22:51:08.663000'), Timestamp('1900-10-09 22:51:10.577000')], [Timestamp('1900-10-09 22:51:26.021000'), Timestamp('1900-10-09 22:51:27.604000')], [Timestamp('1900-10-09 22:51:41.861000'), Timestamp('1900-10-09 22:51:43.875000')], [Timestamp('1900-10-09 22:51:58.131000'), Timestamp('1900-10-09 22:51:59.811000')]]
Index(['time', 'ap', 'type', 'role', 'mac', 'rssi', 'rate', 'sig_mode', 'mcs',
       'bandwidth', 'smoothing', 'not_sounding', 'aggregati